# QuantPilot Research Demo

This notebook demonstrates how to use the QuantPilot modules directly for research.

## Setup
Make sure you're in the project directory and have installed dependencies:
```bash
pip install pandas yfinance scikit-learn
```

In [ ]:
# Import modules
import sys
sys.path.append('.')  # Add project root to path

from modules.data_fetcher import DataFetcher
from modules.backtester import Backtester
from modules.ml_signal import MLSignalGenerator

# Initialize
data_fetcher = DataFetcher()
backtester = Backtester()
ml_signal = MLSignalGenerator()

print("Modules loaded successfully!")

## 1. Fetch Stock Data

In [ ]:
# Fetch 2 years of TSMC data
symbol = "2330.TW"
df = data_fetcher.get_stock_data_df(symbol, period="2y")

print(f"Fetched {len(df)} rows for {symbol}")
df.tail()

## 2. Run Backtest

In [ ]:
# Run MA Crossover strategy with risk management
result = backtester.run(
    stock_df=df,
    strategy="ma_crossover",
    initial_capital=1_000_000,
    params={"short_window": 10, "long_window": 60},
    risk_params={"stop_loss_pct": 0.05, "take_profit_pct": 0.10}
)

print(f"Strategy: {result['strategy']}")
print(f"Return: {result['return_pct']:.2f}%")
print(f"Sharpe Ratio: {result['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {result['max_drawdown_pct']:.2f}%")
print(f"Total Trades: {len(result['trades'])}")

## 3. View Trades

In [ ]:
import pandas as pd

trades_df = pd.DataFrame(result['trades'])
trades_df

## 4. Plot Equity Curve

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(result['equity_curve'])
plt.title('Equity Curve')
plt.xlabel('Days')
plt.ylabel('Portfolio Value')
plt.grid(True, alpha=0.3)
plt.show()

## 5. Train ML Model

In [ ]:
# Train Random Forest model
train_result = ml_signal.train(df, symbol)

if train_result['success']:
    print(f"Model trained successfully!")
    print(f"Accuracy: {train_result['accuracy'] * 100:.1f}%")
else:
    print(f"Training failed: {train_result.get('error')}")

## 6. Get ML Prediction

In [ ]:
# Predict next-day direction
prediction = ml_signal.predict(df, symbol)

if prediction['success']:
    print(f"Signal: {prediction['signal']}")
    print(f"Confidence: {prediction['confidence']}%")
else:
    print(f"Prediction failed: {prediction.get('error')}")

## 7. Feature Importance

In [ ]:
importance = ml_signal.get_feature_importance(symbol)

if importance['success']:
    for feature, imp in importance['importance'].items():
        print(f"{feature}: {imp:.4f}")

## 8. Parameter Optimization

In [ ]:
# Find best MA parameters
param_ranges = {
    "short_window": [5, 10, 20],
    "long_window": [30, 60, 90]
}

results = backtester.optimize(df, "ma_crossover", 1_000_000, param_ranges)

print("Top 3 Parameter Combinations:")
for i, r in enumerate(results[:3], 1):
    print(f"{i}. {r['params']} -> Return: {r['return_pct']:.2f}%")